In [32]:
!pip freeze > requirements.txt

In [33]:
#Required libaries
from datetime import datetime, timezone, timedelta
import csv
import sys, os
import glob
import tweepy
from configparser import ConfigParser
#import tkinter

In [34]:
def config_reader() :
    try:
        config_object = ConfigParser()
        #config_object.read("memoptimize.cfg")
        files = glob.glob("credentials.cfg")
        config_object.read(files)
        return config_object
    except KeyError as k:
        print(f"Key is missing in the config file {k}")
    #arraysize = 1
    #enable_testcase = 'Y'

    #Get the password
    #twitter = config_object["TWITTER"]
    #consumer_key = twitter["consumer_key"]
    #consumer_secret = twitter["consumer_secret"]
    #access_token = twitter["access_token"]
    #access_token_secret = twitter["access_token_secret"]
    

In [35]:
def tweepyAuthenticate():
    config_object = config_reader()
    #auth = tweepy.OAuthHandler("OyEwy3vUmGJUVATYEBIHyh6mB", "4o00Jf50LSzdNTlXLbsAUZVohxA1lfPTxDvHvcEa6tOFORDJug")
    auth = tweepy.OAuthHandler(config_object.get('TWITTER', 'consumer_key'), config_object.get('TWITTER', 'consumer_secret'))
    #auth.set_access_token("3322503788-oduR2Uu5uLrTFx3dPCiCs4E0m8LIN4VfsyGdWTi","R1iQxAkEsA91MehaZMxARSip6Shc8CJeP4sqX7TiPsHFJ")
    auth.set_access_token(config_object.get('TWITTER', 'access_token'), config_object.get('TWITTER', 'access_token_secret'))
    api = tweepy.API(auth)
    # test authentication
    try:
        api.verify_credentials()
        print("Authentication OK")
        return api, config_object
    except:
        print("Error during authentication")

In [36]:
def getLatestFile(folder_path):   
    #folder_path = r'C:\Users\Ron\Desktop\Test'
    file_type = r'\*csv'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

    #print(max_file)
    return max_file

In [37]:
def getSearchDateTime():
    print(os.getcwd())
    #os.chdir('data/twitterFavorite')
    #print(os.getcwd())
    if len(glob.glob(os.getcwd()+r"/data/twitterFavorite/*.csv")) > 0:
        filename = getLatestFile(os.getcwd()+r"/data/twitterFavorite")
        print(filename)
        filetime= filename[filename.rfind('_')+1:filename.rfind('.csv')]
        print(type(datetime.strptime(filetime, '%Y%m%d%H%M')))
        print(datetime.strptime(filetime, '%Y%m%d%H%M').replace(tzinfo=timezone.utc))
        return datetime.strptime(filetime, '%Y%m%d%H%M').replace(tzinfo=timezone.utc)
    else:
        return datetime.now(timezone.utc) - timedelta(hours=1)

In [38]:
def write_csv(list_write):
    keys = list_write[0].keys()
    fileGenerated = datetime.now(timezone.utc).strftime("%Y%m%d%H%M")
    with open(r'data\twitterFavorite\twitter_favorite_'+fileGenerated+'.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(list_write)

### Function favoriteTweet to mark tweets as favorite based on search criteria

In [39]:
def favoriteTweet():
    search_list = ["#python", "#matplotlib", "#pandas", "#streamlit","#seaborn"]
    #search = "#python"
    lang = "en"
    numberOfTweets = 20
    list_write = []
    datetime_object = getSearchDateTime()
    api, _ = tweepyAuthenticate()
    print("date beyond which search to start", datetime_object)
    for search in search_list:
        #print(f"searching tag {search}")
        for tweet in tweepy.Cursor(api.search_tweets, search, lang = lang, result_type = 'recent').items(numberOfTweets):
            try:
                #datetime_str = '2022-12-26 18:00:09+00:00'
                #datetime_object = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S%z')

                if tweet.created_at > datetime_object :
                    #print(tweet.user.screen_name)
                    #print(tweet.favorited)
                    #print("Tweet created time ",tweet.created_at)
                    try:
                        url = tweet.entities['urls'][0]['url']
                        expanded_url = tweet.entities['urls'][0]['expanded_url']
                    except IndexError:
                        url = ""
                        expanded_url = ""
                    #print(tweet)
                    #print(tweet.entities['urls'][0])
                    #print(tweet.entities.urls.expanded_url)
                    #print(str(tweet.created_at))
                    #print("convert to string", tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"))
                    list_write.append({"id":tweet.id,"created_at":tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"),\
                                       "tag":search,"screen_name":tweet.user.screen_name,\
                                       "text":(tweet.text).replace('\n','').encode('utf-8'), "url":url,\
                                       "expanded_url":expanded_url,\
                                       "favorited_at":datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")})
                    tweet.favorite()
                    #print('favorited the tweet')
            except tweepy.TweepyException as e:
                print(e.api_errors)
            except StopIteration:
                break

    #print(list_write)
    write_csv(list_write)

### Below code to follow all IDs following my ID

In [40]:
def followFollowers():
    followers = []
    
    api, config_object = tweepyAuthenticate()
    screen_name = config_object.get('TWITTER', 'screenName')#"mohdatif1987"
    print(f"Followers (accounts who follow {screen_name})")
    for page in tweepy.Cursor(api.get_followers, screen_name=screen_name,
                              count=100).pages(10):
        for user in page:
            name = f"{user.id} - {user.name} (@{user.screen_name})"
            followers.append(name)
            if not user.following:
                print(f"{user.screen_name}")
                try:
                    api.create_friendship(user_id = user.id, follow = False)
                except tweepy.TweepyException as e:
                    print(e.api_errors)
                    continue
                print(f"Started following {user.screen_name}")
        print(len(page))

    print(f"Followers: {len(followers)}")
    #print(followers)

In [41]:
if __name__ == "__main__":
    #favoriteTweet()
    followFollowers()

Authentication OK
Followers (accounts who follow mohdatif1987)
dong_marcelo
Started following dong_marcelo
mindyourownj
[{'code': 160, 'message': "You've already requested to follow mindyourownj."}]
drew_zter
[{'code': 160, 'message': "You've already requested to follow drew_zter."}]
_Iris__Lee
[{'code': 160, 'message': "You've already requested to follow _Iris__Lee."}]
shshp4
[{'code': 160, 'message': "You've already requested to follow shshp4."}]
Livesinashell
[{'code': 160, 'message': "You've already requested to follow Livesinashell."}]
Wilmond621
[{'code': 160, 'message': "You've already requested to follow Wilmond621."}]
saritha928
[{'code': 160, 'message': "You've already requested to follow saritha928."}]
Icy_Bug33
[{'code': 160, 'message': "You've already requested to follow Icy_Bug33."}]
IslasNogueira
[{'code': 160, 'message': "You've already requested to follow IslasNogueira."}]
AJAYSHA09409916
[{'code': 160, 'message': "You've already requested to follow AJAYSHA09409916."}]